# Spatial Reference Systems

### Short Introduction

A Spatial Reference System (SRS) or Coordinate Reference System (CRS) is a mathematical construct that is essential to the discipline of Geography. It has two important roles:
 1. identify unequivocally and with precision the location of a spatial object;
 2. accurately portray spatial objects in maps.

In its most basic form a SRS is composed by two elements:
 1. *Datum* - a sphere or ellipsoid that approximates the shape of the Earth, positioned relative to the latter.
 2. *Cartographic Projection* - a set of mathematical functions that translate locations in the surface of the *datum* into the Cartesian plane.

The World Geodetic System (WGS 84) is a collection of *datums* maintained by the  National
Geospatial Intelligence Agency (NGA) of the USA that approximate the surface of the Earth as a whole. Most GPS or GNSS receivers today report geographic coordinates (latitude and longitude) in reference to one of the WGS 84 *datums*. For global cartography the WGS 84 can be a convenient choice, but for local mapping a bespoke *datum* is  more appropriate in most circumstances. National surveys defined specific *datums* that closely suit their country or region.

![Geodesic Datum](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Gloabl_and_Regional_Ellipsoids.svg/640px-Gloabl_and_Regional_Ellipsoids.svg.png)
*Sketch of geodesic datums positioned relative to the Earth's surface. Source: [Wikipaedia](https://en.wikipedia.org/wiki/Geodetic_datum).*

The Equirectangular and Mercator projections are the most popular today, however, none of them is suited for either local or global cartography. It is always useful to spend some time identifying the most appropriate projection for the work at hand. Cartographic projections introduce errors as they flatten the curved surface of the Earth onto a plane. A balance must be struck between the accuracy of areas, shapes and angles. For global mapping, projections like [Mollweide's Homolographic](https://en.wikipedia.org/wiki/Mollweide_projection), [Eckert IV](https://en.wikipedia.org/wiki/Eckert_IV_projection) or [Goode's Homolosine](https://en.wikipedia.org/wiki/Goode_homolosine_projection) present interesting compromises. For local mapping, the [Stereographic](https://en.wikipedia.org/wiki/Stereographic_projection#Applications_to_other_disciplines), [Lambert's Azimutal Equal-Area](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) and [Gauss-Krüger](https://en.wikipedia.org/wiki/Transverse_Mercator_projection#Ellipsoidal_transverse_Mercator) (*aka* "Transverse Mercator") are popular choices, but many more exist.

![Map Projections](https://upload.wikimedia.org/wikipedia/commons/0/02/Kaardiprojektsiooni_klassid.gif)
*Visual examples of the transformations applied to the *datum* by simple map projections. Source: [Wikipaedia](https://en.wikipedia.org/wiki/Map_projection).*

Always keep in mind one thing: no cartographic projection is able to fully preserve distances correctly. Therefore **avoid computing distances** in the Cartesian plane, as they will be wrong. In small study areas the error might be negligible, but at global or continental scales, even for large countries like Russia, distances computed on the Cartesian plane are significantly off.

### Python libraries

The [PROJ](https://proj.org) library is a cornerstone of FOSS4G, implementing a large number of cartographic projections and most geodetic datums. The list of [cartographic projections implemented by PROJ](https://proj.org/operations/projections/index.html) is a good place to start exploring the different characteristics of each projection. [pyproj](https://pyproj4.github.io/pyproj/stable/) is the native Python interface to PROJ.

In alternative, the GDAL/OGR Python API library includes a dedicated module to handle Spatial Reference Systems (SRS): [osgeo.osr](https://gdal.org/python/osgeo.osr-pysrc.html). It may be convinient in certain cases.

### Create a new Coordinate Reference System object

In `pyproj` the concept of CRS is encapsulated in a class with same name: `CRS`. An object of this class can be parametrised in different ways. The simplest is possibly with a [PROJ4 string](https://proj.org/usage/quickstart.html), a synthetic and expressive string with a set of parameters. This is made with the static method `from_pro4`:

In [1]:
from pyproj import CRS
hammer = CRS.from_proj4("+proj=hammer +lat_0=0 +lon_0=0 +datum=WGS84 +units=m +no_defs +wktext")
hammer

<Projected CRS: +proj=hammer +lat_0=0 +lon_0=0 +datum=WGS84 +units ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: PROJ hammer
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Although simple, PROJ strings leave some room for ambiguity. A more formal and precise way to initialise a `CRS` object is using an [OGC Well Known Text](https://www.opengeospatial.org/standards/wkt-crs) (WKT) definition. This is far more verbose, but also more accurate. The following example initialises a new object with the geographic system based on the WGS 84 datum series:

In [ ]:
geographic = CRS.from_wkt(
	'''GEOGCS[
		"WGS 84",
		DATUM[
			"WGS_1984",
			SPHEROID[
				"WGS 84",6378137,298.257223563,
				AUTHORITY["EPSG","7030"]
			],
			AUTHORITY["EPSG","6326"]
		],
		PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],
		UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],
		AUTHORITY["EPSG","4326"]
	]''')
geographic

Note that no projection was defined, this system is supposed to refer strictly to latitude and longitude. However, most GIS programmes interpret this CRS as including the Equirectangular projection (beware of distance or area computations).

The [European Petroleum Survey Group (EPSG)](http://wiki.gis.com/wiki/index.php/European_Petroleum_Survey_Group) was a scientific body supporting the Petroluem & Gas industry in Europe. It developed an [extensive database](http://www.epsg.org/) of parameters and complete SRS definitions, to help the industry standardise its cartographic processes. The EPSG assigned a unique numerical identifier to each entry in its database, which became rather handy to quickly refer to a specific, well defined, SRS. Most open source geospatial software support the EPSG identifiers as quick reference (some even enforce it). Tools such as [epsg.io](https://epsg.io) provide quick and easy verification of EPSG codes. PROJ and GDAL are no exceptions, and therefore `CRS` objects can too be initialised with an EPSG identifier:

In [ ]:
gauss_krueger_arg = CRS.from_epsg(22174)
gauss_krueger_arg

There are other methods to initialise a `CRS` object, but these are the most common.

It is possible to export a `CRS` into different formats that are simple to read or use by other software. For intance, the `to_wkt` method produces a WKT string:

In [ ]:
print(gauss_krueger_arg.to_wkt(pretty=True))

Finally an example outputting the PROJ4 string. Note how `pyproj` informs on the possible loss of information using this method.

In [ ]:
gauss_krueger_arg.to_proj4()

### Create and use CRS transformations

With the CRSs parametrised, it becomes possible to convert coordinates between different systems. In `pyproj` this is made with the `Transformer` class. It is initialised with an input CRS and an output CRS. The following defines a transformation between geographic coordinates and the Argentinian CRS with the Gauss-Krüger projection:

In [ ]:
from pyproj import Transformer
transformArgentina = Transformer.from_crs(geographic, gauss_krueger_arg)
transformArgentina

*Puente de la Mujer* is one of the modern monuments in Buenos Aires. Relative to the latest WGS84 *datum* its coordinates are approximately 34.61º S and 58.37º W. Let us check its coordinates in the Argentinian system using the `transform` method.

In [ ]:
transformArgentina.transform(-34.61, -58.37) #latitude comes first

Recall from above that in most geographic CRSs (like the WGS84) Latitude comes first, but for the cartesian coordinates, it is Easting first (*xx* axis).

Question: the central meridian of this system is 2º east of Buenos Aires, so why is the easting coordinate so high?


### Practical example

In many cases it is necessary to work with spatial data created by someone else. It is then important to clearly identify the SCRS of such data to make sure it matches the CRS used in the analysis.

The example below opens a dataset with the borders of Argentina and inspects its CRS. This type of data is detailed in the [Vector Data](04-vector-data.ipynb) section.

In [ ]:
import fiona
src_borders = fiona.open('../data/argentina.gpkg')
src_borders.crs_wkt

This dataset includes only geographic coordinates, referring to the WGS84 datum ensemble. The `longlat` parameter means that the [Equirectangular projection](https://en.wikipedia.org/wiki/Equirectangular_projection) is applied when plotting the data or using it directly in spatial analysis. This projection is also known as "Plate Carré".

What does this mean in practice? The best way is to plot the data and see how it looks. A simple way of doing so is with the `matplotlib` library, essentially passing a collection of coordinate pairs to a X-Y plot. In this example the `shape` method from the `shapely` library is used to otain the coordinate pairs from the file opened above.

In [ ]:
from matplotlib import pyplot as plt
from shapely.geometry import shape

feature = next(iter(src_borders))
borders = shape(feature["geometry"])

x,y = borders.geoms[0].exterior.xy

fig = plt.figure(1, dpi=90)
# Without equal aspect the map can be distorted
ax = fig.add_subplot(111, aspect='equal')
ax.plot(x, y, color='#6699cc', alpha=0.7,
    linewidth=2, solid_capstyle='round', zorder=2)
ax.set_title('Argentina - Equirectangular projection')

What if a different CRS is required for analysis? The `transformArgentina` object can be used again, keeping in mind the correct axes order *(latitute, longitude)*.

In [ ]:
coords_transf = transformArgentina.transform(y, x) # Latitude first

`matplotlib` can be used again to visually inspect the outcome.

In [ ]:
fig = plt.figure(1, dpi=90)
ax = fig.add_subplot(111, aspect='equal')
ax.plot(coords_transf[1], coords_transf[0], color='#6699cc', alpha=0.7,
    linewidth=2, solid_capstyle='round', zorder=2)
ax.set_title('Argentina - Gauss-Krüger (datum POSGAR 2007)')

Why aren't the two borders shapes similar? Which of the SRSs would you chose to conduct a spatial analysis in Argentina?

## Conclusions

- Always check the coordinate system of each dataset before using it. Be it in complex analysis or simple mapping.
- Make sure all datasets needed are in the same SRS before combining them.
- Coordinate order is 'wierd' with geographic systems: latitude comes first.
- Take some time to identify the best SRS for the study at hand:
  - Local versus global datum;
  - Projection distortion properties.

---
[<- Geometry](02-geometry.ipynb) | [Vector data ->](04-vector-data.ipynb)
